# Homework 10: `SQL`

## Due Date:  Thursday, November 16th at 11:59 PM

You will create a database of the NASA polynomial coefficients for each specie.

**Please turn in your database with your `Jupyter` notebook!**

# Question 1: Convert XML to a SQL database

Create two tables named `LOW` and `HIGH`, each corresponding to data given for the low and high temperature range.
Each should have the following column names:

- `SPECIES_NAME`
- `TLOW`
- `THIGH`
- `COEFF_1`
- `COEFF_2`
- `COEFF_3`
- `COEFF_4`
- `COEFF_5`
- `COEFF_6`
- `COEFF_7`

Populate the tables using the XML file you created in last assignment. If you did not complete the last assignment, you may also use the `example_thermo.xml` file.

`TLOW` should refer to the temperature at the low range and `THIGH` should refer to the temperature at the high range.  For example, in the `LOW` table, $H$ would have `TLOW` at $200$ and `THIGH` at $1000$ and in the `HIGH` table, $H$ would have `TLOW` at $1000$ and `THIGH` at $3500$.

For both tables, `COEFF_1` through `COEFF_7` should be populated with the corresponding coefficients for the low temperature data and high temperature data.

In [2]:
import x as ET
tree=ET.parse("example_thermo.xml")
elementroot=tree.getroot()
import sqlite3
import pandas as pd
db = sqlite3.connect('thermo.sqlite')
cursor = db.cursor()

In [4]:
cursor.execute("DROP TABLE IF EXISTS low")
cursor.execute("DROP TABLE IF EXISTS high")
cursor.execute("PRAGMA foreign_keys=1")
cursor.execute('''CREATE TABLE low (
               species_name TEXT PRIMARY KEY NOT NULL, 
               tlow INT, 
               thigh INT, 
               coeff_1 FLOAT,
               coeff_2 FLOAT,
               coeff_3 FLOAT,
               coeff_4 FLOAT,
               coeff_5 FLOAT,
               coeff_6 FLOAT,
               coeff_7 FLOAT)''')

cursor.execute('''CREATE TABLE high (
               species_name TEXT PRIMARY KEY NOT NULL, 
               tlow INT, 
               thigh INT, 
               coeff_1 FLOAT,
               coeff_2 FLOAT,
               coeff_3 FLOAT,
               coeff_4 FLOAT,
               coeff_5 FLOAT,
               coeff_6 FLOAT,
               coeff_7 FLOAT)''')

In [7]:
elements=elementroot.find('phase').find("speciesArray").text.strip().split(" ")


['H', 'O', 'OH', 'H2', 'O2', 'H2O', 'HO2', 'H2O2', 'N2', 'Hp', 'Op']


In [13]:
speciesroot=elementroot.find("speciesData")
for specie in speciesroot:
    species_name=specie.attrib['name']
    C=specie.find("thermo").findall("NASA")
    for i,cc in enumerate(C):
        coefficients=cc.find("floatArray").text.strip().split(" ")
        thigh=cc.attrib['Tmax']
        tlow=cc.attrib['Tmin']
        coeff_1=coefficients[0]
        coeff_2=coefficients[1] 
        coeff_3=coefficients[2]
        coeff_4=coefficients[3]
        coeff_5=coefficients[4]
        coeff_6=coefficients[5]
        coeff_7=coefficients[6]
        
        vals_to_insert = (species_name, tlow, thigh, 
                           coeff_1, coeff_2, coeff_3, coeff_4, coeff_5, coeff_6, coeff_7)

        if i==0:
            cursor.execute('''INSERT INTO low (species_name, tlow, thigh, 
                           coeff_1, coeff_2, coeff_3, coeff_4, coeff_5, coeff_6, coeff_7) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)
        else:
            cursor.execute('''INSERT INTO high (species_name, tlow, thigh, 
                           coeff_1, coeff_2, coeff_3, coeff_4, coeff_5, coeff_6, coeff_7) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)
        

In [16]:
db.commit()

In [71]:
high_cols = [col[1] for col in cursor.execute("PRAGMA table_info(HIGH)")]
query = '''SELECT * FROM HIGH'''
viz_tables(high_cols, query)

,species_name,tlow,thigh,coeff_1,coeff_2,coeff_3,coeff_4,coeff_5,coeff_6,coeff_7
0,H,1000,5000,"2.50000000E+00,","0.00000000E+00,","0.00000000E+00,","0.00000000E+00,","0.00000000E+00,","2.54716270E+04,",-0.460118
1,O,1000,5000,"2.54205966E+00,","-2.75506191E-05,","-3.10280335E-09,","4.55106742E-12,","-4.36805150E-16,","2.92308027E+04,",4.920308
2,OH,1000,6000,"2.86472886E+00,","1.05650448E-03,","-2.59082758E-07,","3.05218674E-11,","-1.33195876E-15,","3.68362875E+03,",5.701641
3,H2,1000,5000,"2.99142337E+00,","7.00064411E-04,","-5.63382869E-08,","-9.23157818E-12,","1.58275179E-15,","-8.35033997E+02,",-1.355110
4,O2,1000,5000,"3.69757819E+00,","6.13519689E-04,","-1.25884199E-07,","1.77528148E-11,","-1.13643531E-15,","-1.23393018E+03,",3.189166
5,H2O,1000,5000,"2.67214561E+00,","3.05629289E-03,","-8.73026011E-07,","1.20099639E-10,","-6.39161787E-15,","-2.98992090E+04,",6.862817
6,HO2,1000,3500,"4.01721090E+00,","2.23982013E-03,","-6.33658150E-07,","1.14246370E-10,","-1.07908535E-14,","1.11856713E+02,",3.785102
7,H2O2,1000,5000,"4.57316685E+00,","4.33613639E-03,","-1.47468882E-06,","2.34890357E-10,","-1.43165356E-14,","-1.80069609E+04,",0.501137
8,N2,1000,5000,"0.02926640E+02,","0.01487977E-01,","-0.05684761E-05,","0.01009704E-08,","-0.06753351E-13,","-0.09227977E+04,",5.980528
9,Hp,1000,5000,"1.64243522,","2.89759059E-04,","0.00000000E+00,","0.00000000E+00,","0.00000000E+00,","-4.69510297E+00,",-11.148334


# Question 2: `WHERE` Statements

1. Write a `Python` function `get_coeffs` that returns an array of 7 coefficients.  
   
   The function should take in two parameters: 1.) `species_name` and 2.) `temp_range`, an indicator variable ('low' or 'high') to indicate whether the coefficients should come from the low or high temperature range.  
   The function should use `SQL` commands and `WHERE` statements on the table you just created in Question 1 (rather than taking data from the XML directly).
```python
def get_coeffs(species_name, temp_range):
    ''' Fill in here'''
    return coeffs
```

2. Write a python function `get_species` that returns all species that have a temperature range above or below a given value. The function should take in two parameters: 1.) `temp` and 2.) `temp_range`, an indicator variable ('low' or 'high').

  When temp_range is 'low', we are looking for species with a temperature range lower than the given temperature, and for a 'high' temp_range, we want species with a temperature range higher than the given temperature.

  This exercise may be useful if different species have different `LOW` and `HIGH` ranges.

  And as before, you should accomplish this through `SQL` queries and where statements.

```python
def get_species(temp, temp_range):
    ''' Fill in here'''
    return coeffs
```

In [72]:
def get_coeffs(species_name, temp_range):
    if temp_range=="low":
        function = '''SELECT coeff_1,coeff_2,coeff_3,coeff_4,coeff_5,coeff_6,coeff_7 FROM low WHERE species_name=?'''
        coeffs=cursor.execute(function,(species_name)).fetchone()
  
    else:
        function = '''SELECT coeff_1,coeff_2,coeff_3,coeff_4,coeff_5,coeff_6,coeff_7 FROM high WHERE species_name=?'''
        coeffs=cursor.execute(function,(species_name)).fetchone()

    return coeffs

In [74]:
get_coeffs("H","high")

('2.50000000E+00,',
 '0.00000000E+00,',
 '0.00000000E+00,',
 '0.00000000E+00,',
 '0.00000000E+00,',
 '2.54716270E+04,',
 -0.460117638)

In [84]:
def get_species(temp, temp_range):
    ''' Fill in here'''
    if temp_range=="low":
        
        function = '''SELECT species_name FROM low WHERE tlow<?'''
        species=cursor.execute(function,(temp_range,)).fetchall()

    else:
        function = '''SELECT species_name FROM high WHERE thigh>?'''
        species=cursor.execute(function,(temp_range,)).fetchall()
    return species

In [88]:
get_species("low",1)

[('H',),
 ('O',),
 ('OH',),
 ('H2',),
 ('O2',),
 ('H2O',),
 ('HO2',),
 ('H2O2',),
 ('N2',),
 ('Hp',),
 ('Op',)]

In [99]:
all_cols = [col[1] for col in cursor.execute("PRAGMA table_info(ALL_TEMPS)")]
query = '''SELECT species_name FROM low WHERE tlow<?'''
viz_tables(all_cols,(query,(1,)))

ValueError: operation parameter must be str

# Question 3: `JOIN` STATEMENTS

Create a table named `ALL_TEMPS` that has the following columns:

- `SPECIES_NAME`
- `TEMP_LOW`
- `TEMP_HIGH`

This table should be created by joining the tables `LOW` and `HIGH` on the value `SPECIES_NAME`.

1. Write a `Python` function `get_range` that returns the range of temperatures for a given species_name.

The range should be computed within the `SQL` query (i.e. you should subtract within the `SELECT` statement in the `SQL` query).
```python
def get_range(species_name):
    '''Fill in here'''
    return range
```

Note that `TEMP_LOW` is the lowest temperature in the `LOW` range and `TEMP_HIGH` is the highest temperature in the `HIGH` range.

In [90]:
def get_range(species_name):
    function = '''SELECT tlow FROM low WHERE species_name=?'''
    temp_low=cursor.execute(function, (species_name,)).fetchall()[0]

    function = '''SELECT thigh FROM high WHERE species_name=?'''
    temp_high=cursor.execute(function,(species_name,)).fetchall()[0]
    
    return (temp_low[0],temp_high[0])

In [91]:
get_range("HO2")

(200, 3500)

In [96]:
cursor.execute("DROP TABLE IF EXISTS all_temps")
cursor.execute('''CREATE TABLE all_temps(
               species_name TEXT PRIMARY KEY NOT NULL, 
               temp_low INT, 
               temp_high INT)''')

function = '''SELECT species_name FROM low'''
species=cursor.execute(function).fetchall()
for specie in species:
    temp_low,temp_high=get_range(specie[0])
    print(specie,temp_low,temp_high)
    cursor.execute('''INSERT INTO all_temps(species_name, temp_low, temp_high) 
                    VALUES(?, ?, ?)''',(specie[0],temp_low,temp_high))
    
    

('H',) 300 5000
('H2',) 300 5000
('H2O',) 300 5000
('H2O2',) 300 5000
('HO2',) 200 3500
('Hp',) 300 5000
('N2',) 300 5000
('O',) 300 5000
('O2',) 300 5000
('OH',) 200 6000
('Op',) 300 5000


In [97]:
cursor.execute("DROP TABLE IF EXISTS ALL_TEMPS")
cursor.execute('''
CREATE TABLE ALL_TEMPS AS
    SELECT HIGH.SPECIES_NAME, HIGH.THIGH AS TEMP_HIGH, LOW.TLOW AS TEMP_LOW
    FROM HIGH
        JOIN LOW
            ON HIGH.SPECIES_NAME = LOW.SPECIES_NAME''')
db.commit()

In [98]:
all_cols = [col[1] for col in cursor.execute("PRAGMA table_info(ALL_TEMPS)")]
query = '''SELECT * FROM ALL_TEMPS'''
viz_tables(all_cols, query)

,species_name,TEMP_HIGH,TEMP_LOW
0,H,5000,300
1,O,5000,300
2,OH,6000,200
3,H2,5000,300
4,O2,5000,300
5,H2O,5000,300
6,HO2,3500,200
7,H2O2,5000,300
8,N2,5000,300
9,Hp,5000,300
